<a href="https://colab.research.google.com/github/sat0urn/CV_Assignment_1/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading Dataset**

In [ ]:
import numpy as np
import os
import cv2
import random
import math

# Loading Dataset

def load_dataset(data_dir):
   classes = os.listdir(data_dir)
   images = []
   labels = []
   for class_name in classes:
       class_dir = os.path.join(data_dir, class_name)
       for image_file in os.listdir(class_dir):
           image_path = os.path.join(class_dir, image_file)
           image = cv2.imread(image_path)
           image = cv2.resize(image, (128, 128))
           images.append(image)
           labels.append(class_name)
   return np.array(images), np.array(labels)

data_dir = '/content/drive/MyDrive/Colab Notebooks/Agricultural-crops'
images, labels = load_dataset(data_dir)
print(labels)

['Cucumber' 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber'
 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber'
 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber'
 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber'
 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber' 'Cucumber'
 'Cucumber' 'chilli' 'chilli' 'chilli' 'chilli' 'chilli' 'chilli' 'chilli'
 'chilli' 'chilli' 'chilli' 'chilli' 'chilli' 'chilli' 'chilli' 'chilli'
 'chilli' 'chilli' 'chilli' 'chilli' 'chilli' 'chilli' 'chilli' 'chilli'
 'clove' 'clove' 'clove' 'clove' 'clove' 'clove' 'clove' 'clove' 'clove'
 'clove' 'clove' 'clove' 'clove' 'clove' 'clove' 'clove' 'clove' 'clove'
 'clove' 'clove' 'clove' 'clove' 'clove' 'clove' 'clove' 'clove' 'clove'
 'clove' 'clove' 'clove' 'banana' 'banana' 'banana' 'banana' 'banana'
 'banana' 'banana' 'banana' 'banana' 'banana' 'banana' 'banana' 'banana'
 'banana' 'banana' 'banana' 'banana' 'banana' 'banana' 'banana' 'banana'
 'banana'